In [1]:
import pandas as pd
import numpy as np
import nflgame

In [3]:
help(nflgame.games)

Help on function games in module nflgame:

games(year, week=None, home=None, away=None, kind='REG', started=False)
    games returns a list of all games matching the given criteria. Each
    game can then be queried for player statistics and information about
    the game itself (score, winner, scoring plays, etc.).
    
    As a special case, if the home and away teams are set to the same team,
    then all games where that team played are returned.
    
    The kind parameter specifies whether to fetch preseason, regular season
    or postseason games. Valid values are PRE, REG and POST.
    
    The week parameter is relative to the value of the kind parameter, and
    may be set to a list of week numbers.
    In the regular season, the week parameter corresponds to the normal
    week numbers 1 through 17. Similarly in the preseason, valid week numbers
    are 1 through 4. In the post season, the week number corresponds to the
    numerical round of the playoffs. So the wild card r

In [177]:
games09 = nflgame.games(2009, week=[1])

In [178]:
games09 = [games09[0]]

In [179]:
games09

In [180]:
g1 = nflgame.combine_plays(games09)

In [181]:
g1

In [182]:
data = []
for p in g1:
    data.append(p.data)

In [183]:
data

[{u'desc': u'R.Bironas kicks 67 yards from TEN 30 to PIT 3. S.Logan to PIT 42 for 39 yards (M.Griffin).',
  u'down': 0,
  u'note': u'KICKOFF',
  u'players': {u'00-0020962': [{u'clubcode': u'TEN',
     u'playerName': u'R.Bironas',
     u'sequence': 1,
     u'statId': 41,
     u'yards': 67}],
   u'00-0025406': [{u'clubcode': u'TEN',
     u'playerName': u'M.Griffin',
     u'sequence': 3,
     u'statId': 79,
     u'yards': 0}],
   u'00-0026491': [{u'clubcode': u'PIT',
     u'playerName': u'S.Logan',
     u'sequence': 2,
     u'statId': 45,
     u'yards': 39}]},
  u'posteam': u'TEN',
  u'qtr': 1,
  u'sp': 0,
  u'time': u'15:00',
  u'ydsnet': 0,
  u'ydstogo': 0,
  u'yrdln': u'TEN 30'},
 {u'desc': u'(14:53) B.Roethlisberger pass short left to H.Ward to PIT 47 for 5 yards (C.Hope).',
  u'down': 1,
  u'note': None,
  u'players': {u'00-0017162': [{u'clubcode': u'PIT',
     u'playerName': u'H.Ward',
     u'sequence': 3,
     u'statId': 21,
     u'yards': 5},
    {u'clubcode': u'PIT',
     u'playe

In [190]:
for d in data:
    print d.keys()
    print("---")

[u'ydstogo', u'note', u'qtr', u'yrdln', u'sp', u'down', u'players', u'time', u'ydsnet', u'posteam', u'desc']
---
[u'ydstogo', u'note', u'qtr', u'yrdln', u'sp', u'down', u'players', u'time', u'ydsnet', u'posteam', u'desc']
---
[u'ydstogo', u'note', u'qtr', u'yrdln', u'sp', u'down', u'players', u'time', u'ydsnet', u'posteam', u'desc']
---
[u'ydstogo', u'note', u'qtr', u'yrdln', u'sp', u'down', u'players', u'time', u'ydsnet', u'posteam', u'desc']
---
[u'ydstogo', u'note', u'qtr', u'yrdln', u'sp', u'down', u'players', u'time', u'ydsnet', u'posteam', u'desc']
---
[u'ydstogo', u'note', u'qtr', u'yrdln', u'sp', u'down', u'players', u'time', u'ydsnet', u'posteam', u'desc']
---
[u'ydstogo', u'note', u'qtr', u'yrdln', u'sp', u'down', u'players', u'time', u'ydsnet', u'posteam', u'desc']
---
[u'ydstogo', u'note', u'qtr', u'yrdln', u'sp', u'down', u'players', u'time', u'ydsnet', u'posteam', u'desc']
---
[u'ydstogo', u'note', u'qtr', u'yrdln', u'sp', u'down', u'players', u'time', u'ydsnet', u'postea